In [1]:
import os
from pathlib import Path
import json

import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import Sequential, GCNConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool

import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

In [2]:
project_path = str(Path(os.getcwd()).parent.absolute())
print(project_path)
os.chdir(project_path)


/Users/maximebonnin/Notebooks/3A_Notebook/INF554/INF554_Kaggle_Project


In [3]:
def convert_string_to_tensor(string):
    return string.replace('[', '').replace(']', '').replace(' ', '').split(',')

def get_x(vectors):
    x = np.zeros((len(vectors), 201))
    for i,vector in tqdm(enumerate(vectors)):
        print(int(vector))
        x[i,0] = int(vector)
        x[i,1:] = np.array(convert_string_to_tensor(vectors[vector]['vector']))
        #x.append(convert_string_to_tensor(vectors[vector]['vector']))
        #x[-1].append(int(vector))
        #print(x)
        if i==10:
            break
    return np.array(x, dtype=np.float32)


In [27]:
df_train = pd.read_csv('data/train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]

# read test data
df_test = pd.read_csv('data/test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]

# load the graph    
G = nx.read_edgelist('data/coauthorship.edgelist', delimiter=' ', nodetype=int)

Exception ignored in: <bound method tqdm.__del__ of <tqdm.notebook.tqdm_notebook object at 0x146666198>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


In [59]:
edge_index = np.array(G.edges())
edge_index = np.moveaxis(edge_index, 0, 1)
edge_index.shape

(2, 1718164)

In [4]:
with open("Vectors_and_clusters.json") as file:
    vectors = json.load(file)

In [5]:
x = get_x(vectors)
#del vectors

10it [00:00, 824.55it/s]


3603
7383
14309
15548
15883
43287
43295
49226
49575
54153
69374


In [11]:
with open("data/abstracts.txt", encoding='utf-8') as file:
    for i in range(50):
        print(file.readline())
    #abstracts = file.readlines()


3603----{"IndexLength":122,"InvertedIndex":{"In":[0],"this":[1],"paper,":[2],"we":[3,98],"describe":[4],"a":[5,16,41,58,62,67,74,108],"new":[6,17],"bitmap":[7,42,54,69,76],"indexing":[8,43],"technique":[9],"to":[10,73],"cluster":[11],"XML":[12,14,34,63],"documents.":[13],"is":[15,71],"standard":[18,89],"for":[19,60],"exchanging":[20],"and":[21,38,49,56,91,104,115],"representing":[22],"information":[23],"on":[24,95,107],"the":[25,47,85,101,117],"Internet.":[26],"Documents":[27],"can":[28,111],"be":[29,112],"hierarchically":[30],"represented":[31,37],"by":[32],"XML-elements.":[33],"documents":[35],"are":[36],"indexed":[39],"using":[40],"technique.":[44],"We":[45,80],"define":[46,81,100],"similarity":[48],"popularity":[50],"operations":[51,106],"available":[52],"in":[53,84],"indexes":[55],"propose":[57],"method":[59],"partitioning":[61],"document":[64,120],"set.":[65],"Furthermore,":[66],"2-dimensional":[68],"index":[70],"extended":[72],"3dimensional":[75],"index,":[77],"called":[78],"Bit

In [31]:
x.shape

(624181, 201)

In [29]:
np.save('data/x.npy', x)

In [60]:
data = Data(x=torch.tensor(x[:,1:], dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long))

In [62]:
print(data.num_nodes)
print(data.num_features)
data.num_edges

624181
200


1718164

In [64]:
loader = DataLoader([data], batch_size=32)

In [65]:
model = Sequential('x, edge_index, batch', [
    (Dropout(p=0.5), 'x -> x'),
    (GCNConv(data.num_features, 64), 'x, edge_index -> x1'),
    ReLU(inplace=True),
    (GCNConv(64, 64), 'x1, edge_index -> x2'),
    ReLU(inplace=True),
    (lambda x1, x2: [x1, x2], 'x1, x2 -> xs'),
    (JumpingKnowledge("cat", 64, num_layers=2), 'xs -> x'),
    (global_mean_pool, 'x, batch -> x'),
    Linear(2 * 64, 1),
])